In [429]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs

In [430]:
sport_dict = {
         '田徑'   : 'A',
         '足球'   : 'B',
         '籃球'   : 'C',
         '排球'   : 'D',
         '網球'   : 'E',
         '棒球'   : 'F',
         '壘球'   : 'G',
         '橄欖球' : 'H',
         '羽球'   : 'S',
         '桌球'   : 'T'
    }

In [490]:
def get_sport_raw(date:str, sport_in:str):
    session = requests.session()
    url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=ste11211'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        # 'Cookie': 'PHPSESSID=1aea228467450d51917fc7f8f835ff86',
        'Cookie': 'PHPSESSID=034a060d5072fe85a95b24e83d0bf2f9'
        # TODO: 把這邊的 Cookie 改成 CookieJar
        # 'X-Requested-With': 'XMLHttpRequest'
    }

    payload = {
        'sport': sport_dict[sport_in],
        'sdate': date,
        'c': 'ste11211',
        'm': 'read'
    }


    login = requests.post(url=url, headers=headers, data=payload)
    # volleyball = session.post(url=url_get, headers=headers, data=payload)
    print(login.text)
    # print(volleyball.text)
    # with open('test.html', 'w', encoding='utf-8') as fp:
    #     fp.write(volleyball.text)

    # replace is for '體育室 <br/>(數學系)' (in raw)
    # but in '體育室 <br>(數學系)' (in html)
    # return bs(volleyball.text.replace('<br>', ''), 'html.parser')

In [491]:
get_sport_raw('20201222', '排球')

<div class="site_table_div" >	
	<h2>
		2020年12月22日 	</h2>
	<table class="gray_table2">
		<thead>
			<tr>
				<!-- 顯示欄位標題 -->
				<th style='width:10%;'></th>
				<th style='width:5%;'>06:00</br>07:00</th><th style='width:5%;'>07:00</br>08:00</th><th style='width:5%;'>08:00</br>09:00</th><th style='width:5%;'>09:00</br>10:00</th><th style='width:5%;'>10:00</br>11:00</th><th style='width:5%;'>11:00</br>12:00</th><th style='width:5%;'>12:00</br>13:00</th><th style='width:5%;'>13:00</br>14:00</th><th style='width:5%;'>14:00</br>15:00</th><th style='width:5%;'>15:00</br>16:00</th><th style='width:5%;'>16:00</br>17:00</th><th style='width:5%;'>17:00</br>18:00</th><th style='width:5%;'>18:00</br>19:00</th><th style='width:5%;'>19:00</br>20:00</th><th style='width:5%;'>20:00</br>21:00</th>				
			</tr>
		</thead>
		<tbody>
			<!-- 顯示資料 -->
			<tr><th>光復低場排球1</br>D1001</th><td colspan='6' class="cell3">H2統計系</td><td colspan='6' class="cell4">體育室  </td><td colspan='3' class="cell3">Q1微電所</td></t

In [439]:
def retreive_form(soup):
    if not isinstance(soup, type(bs())):
        raise TypeError("soup must be a " + repr(type(bs())))
    else:
        form =[]
        row = []
        counter = 0
        for content in new_s.find_all('td'):
            # print(content, '\t\t', repr(content.string))
            if content.string not in ('\xa0\xa0', '可借用', '不可借用', '已借用'):
                if counter == 3:
                    counter = 0
                    form.append(row)
                    row = []
                row.append(content.string)
                counter += 1
    
    return form

In [440]:
from datetime import datetime, timedelta
def gen_dates(start:str, end:str):
    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')
    serial = []
    if (end_date - start_date).days >= 0:
        for i in range((end_date - start_date).days+1):
            tmp = start_date + timedelta(days=i)
            serial.append('{0:>{3}4}{1:>{3}2}{2:>{3}2}'.format(tmp.year, tmp.month, tmp.day, 0))
            
    return serial

In [442]:
# output to html file
with open('test1.html', 'w', encoding='utf-8') as fp:
    fp.write(soup.prettify())

In [443]:
new_s = get_sport_raw('20201221', '排球')
t = retreive_form(new_s)
print(t)
# print(type(new_s))
# new_s = new_s.replace('<br/', '')
# print(repr(new_s))

[[None, '\n\t\t\t\t\t\t\t\t\t\t\t\t\t帳號\n\t\t\t\t\t\t\t\t\t\t\t\t', None], ['\n\t\t\t\t\t\t\t\t\t\t\t\t\t密碼\n\t\t\t\t\t\t\t\t\t\t\t\t', None, '\n\t\t\t\t\t\t\t\t\t\t\t\t\t驗證碼\n\t\t\t\t\t\t\t\t\t\t\t\t']]


In [423]:
gen_dates('20201202', '20201205')

['20201202', '20201203', '20201204', '20201205']

In [396]:
(goal2 - goal).days

-48

In [256]:
soup = bs(result.text, 'html.parser')
# soup

In [405]:
class a:
    value = 3

In [407]:
def add(t):
    print('before =', t.value)
    t.value = 0
    print('after =', t.value)
    return t 